In [ ]:
!wget https://raw.githubusercontent.com/wgruen/lizard/master/mount_gdrive.ipynb

#!ls
%run mount_gdrive.ipynb


--2022-02-14 00:44:26--  https://raw.githubusercontent.com/wgruen/lizard/master/mount_gdrive.ipynb
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10889 (11K) [text/plain]
Saving to: ‘mount_gdrive.ipynb’

mount_gdrive.ipynb  100%[===================>]  10.63K  --.-KB/s    in 0s      

2022-02-14 00:44:26 (47.2 MB/s) - ‘mount_gdrive.ipynb’ saved [10889/10889]

Mounted at /content/drive
drive  mount_gdrive.ipynb  sample_data
/content
physical devises
[]
tensorflow version
2.7.0
keras version
2.7.0
/bin/bash: nvidia-smi: command not found
Your runtime has 13.6 gigabytes of available RAM

Not using a high-RAM runtime


In [ ]:
# run a notebook
exit()  # exit runtime, so it restarts the runtime session


!wget -O get_gpu.ipynb https://raw.githubusercontent.com/wgruen/lizard/master/get_gpu.ipynb

#!ls
%run get_gpu.ipynb

%cd "/content/drive/MyDrive/colab_wolfs_git_clones/lizard"
#!ls 
#!pwd

%cd "nmf"



--2022-02-14 00:45:32--  https://raw.githubusercontent.com/wgruen/lizard/master/get_gpu.ipynb
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10864 (11K) [text/plain]
Saving to: ‘get_gpu.ipynb’

get_gpu.ipynb       100%[===================>]  10.61K  --.-KB/s    in 0s      

2022-02-14 00:45:32 (91.8 MB/s) - ‘get_gpu.ipynb’ saved [10864/10864]

physical devises
[]
tensorflow version
2.7.0
keras version
2.7.0
/bin/bash: nvidia-smi: command not found


SystemError: ignored

/content/drive/MyDrive/colab_wolfs_git_clones/lizard
/content/drive/MyDrive/colab_wolfs_git_clones/lizard/nmf


In [ ]:
from numpy import exp, array, random, dot, round
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 
import os

!pip install reportlab 
from reportlab.pdfgen import canvas
from reportlab.platypus import *
from reportlab.lib import colors
from reportlab.lib.pagesizes import letter
from reportlab.lib.styles import ParagraphStyle, getSampleStyleSheet

styles = getSampleStyleSheet()

#current dir
cwd = os.getcwd()
print(cwd)

! ls

     |████████████████████████████████| 2.8 MB 7.6 MB/s 
/content/drive/MyDrive/colab_wolfs_git_clones/lizard/nmf
data  news_charaterization.ipynb  news_charaterization.py  nmf_newspaper.ipynb


### Read the data 

In [ ]:
with open('./data/nyt_data.txt') as f:
    documents = f.readlines()
documents = [x.strip().strip('\n').strip("'") for x in documents] 

### Read the Vocabulary (dictionary)

In [ ]:
with open('data/nyt_vocab.dat') as f:
    vocabs = f.readlines()
vocabs = [x.strip().strip('\n').strip("'") for x in vocabs] 

### Create matrix for documents

In [ ]:
'''create matrix X'''
numDoc = 8447
numWord = 3012 
X = np.zeros([numWord,numDoc])

for col in range(len(documents)):
    for row in documents[col].split(','):
        X[int(row.split(':')[0])-1,col] = int(row.split(':')[1])

In [ ]:
'''randomly initialize W and H with nonnegative values'''
rank = 25
T = 100
W = np.zeros([numWord,rank])
H = np.zeros([rank,numDoc])

for row in range(numWord):
    W[row] = np.random.rand(rank)
for row in range(rank):
    H[row] = np.random.rand(numDoc)
    
    
    

In [ ]:
'''setting divergence penalty''' #iterate values in H, then in W
d_iter = np.zeros(100)

for iteration in range(100):
    
    '''iterate all values in H'''
    m1 = np.dot(W.T,X)
    m2 = np.dot(W,H)
    m3 = np.dot(m2.T,W)
    second = np.divide(m1,m3.T + 0.0000000000000001)

    for k in range(rank):
        for j in range(numDoc):
            H[k,j] = np.multiply(H[k,j], second[k,j])
    
    '''iterate all values in W'''
    n1 = np.dot(H,X.T)
    n2 = np.dot(W,H)
    n3 = np.dot(n2,H.T)
    third = np.divide(n1.T,n3 + 0.0000000000000001)

    for i in range(numWord):
        for k in range(rank):
            W[i,k] = np.multiply(W[i,k], third[i,k])
        
    '''plot objective function'''
#     D = np.multiply(X, np.log(1/(n2 + 0.0000000000000001))) + n2
#     d_iter[iteration] = np.sum(D)
    D = np.multiply(X,np.log(n2+0.0000000000000001)) - n2
    d_iter[iteration] = -np.sum(D)

fig= plt.figure(figsize = (15,6))
ax = fig.add_subplot(1,1,1)
ax.plot(range(100),d_iter[:100])
plt.title('Plot of divergence objective in 100 iterations')
plt.ylabel('$D(X||WH)$')
plt.xlabel('iteration $t$')
plt.show()


# ### b. Ten words with the largest weight.

# In[502]:

'''normalize each column to sum to zero'''
W_normed = W / np.sum(W,axis=0)


# In[511]:

'''for each column of W, list the 10 words having the largest weight and show the weight'''
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 120)    
vList = []


if not os.path.exists("output"):
    os.mkdir("output")
doc_summary  = SimpleDocTemplate("output/news_charaterization_summary.pdf"+ "_summary.pdf", pagesize=letter)
element = []
header = Paragraph("\nSummary of Analysis Run", styles["Heading1"])
element.append(header)
    


for topic in range(rank):
    v = pd.DataFrame(vocabs)
    v[1] = W_normed[:,topic].round(6)
    v = v.sort_values([1, 0], ascending=[0,1]).rename(index=int, columns={0: "Topic {}".format(topic+1), 1: "Weight"}).head(10)
    v = v.reset_index(drop=True)
    vList.append(v)
    
#    lista = [df.columns[:,].values.astype(str).tolist()] + df.values.tolist()
#    t1 = Table(lista)        
#    element.append(t1)
#    element.append(v)
     

    
for num in [5,10,15,20,25]:
    print('\n',(pd.concat(vList[num-5:num], axis=1)),'\n')
    print("lineline")
    print(vList[num-5:num])
    print("linelineline")
    t1 = Table(vList[num-5:num])  
    element.append(t1)
    t11 = vList[num-5:num]
    print("t11")
    print(t11)
    element.append(Table(t11))
    
    
doc_summary.build(element)
    
# sort by standard deviation  
#header = Paragraph("\nSorted by Interquartile Range", styles["Heading2"])Analysis
#element.append(header)
    

#df = df.sort_values(by=["interquartile_range"])
#    lista = [df.columns[:,].values.astype(str).tolist()] + df.values.tolist()
#    t1 = Table(lista)        
#    element.append(t1)